# ETL Punti di interesse

### ETL Municipi

Carichiamo il file .shp (shape file) e scriviamolo in Clean in formato geojson

In [ ]:
import fiona
import fiona.transform
import json

PATH_MUNICIPI = "./Data/Raw/Punti_di_interesse/Municipi/Municipi.shp"
PATH_MUNICIPI_JSON = "./Data/Clean/Punti_di_interesse/Municipi.geojson"

src_crs = 'EPSG:6707'
dst_crs = 'EPSG:4326'

with fiona.open(PATH_MUNICIPI, 'r') as src:
    with fiona.open(PATH_MUNICIPI_JSON, 'w', driver='GeoJSON', crs=src.crs, schema=src.schema) as dst:
        for feature in src:
            feature['geometry'] = fiona.transform.transform_geom(
                src_crs,
                dst_crs,
                feature['geometry']
            )
            dst.write(feature)

# Solo per una corretta indentazione
with open(PATH_MUNICIPI_JSON, 'r') as file:
    file_json = json.load(file)

with open(PATH_MUNICIPI_JSON, 'w') as file:
    json.dump(file_json, file, indent=4)

### ETL PARCHI

Carichiamo il file .shp (shape file) e scriviamolo in Clean in formato geojson

In [1]:
import fiona
import json

PATH_PARCHI_SHP = "./Data/Raw/Punti_di_interesse/Parchi/Parchi_WGS84_Milano_1.shp"
PATH_PARCHI_SHP_JSON = "./Data/Clean/Punti_di_interesse/parchi.geojson"

with fiona.open(PATH_PARCHI_SHP, 'r') as src:
    with fiona.open(PATH_PARCHI_SHP_JSON, 'w', driver='GeoJSON', crs=src.crs, schema=src.schema) as dst:
        for feature in src:
            if feature["geometry"]["type"] == "Polygon":
                dst.write(feature)

# Solo per una corretta indentazione
with open(PATH_PARCHI_SHP_JSON, 'r') as file:
    file_json = json.load(file)
    
with open(PATH_PARCHI_SHP_JSON, 'w') as file:
    json.dump(file_json, file, indent=4)